# NU.nl comment harvester

You need the Python library selenium (pip install selenium) and the geckodriver: https://github.com/mozilla/geckodriver/releases

Instructions:

1. collect the list of relevant urls in a text file (see below)
2. run the first code block to load library imports and read the url file
3. set the start id in the for loop in the fifth code block (first id is 0) (?)
4. run code block 6 and wait
5. the comments are stored in a set of files `[category]-[article_id].csv`

Collect list of relevant URLs:

1. run the second code block to open https://www.nu.nl/tag/Coronavirus
2. run the third code block to get the desired time period on the page
3. copy-paste from browser window to LibreOffice Writer
4. save as Flat ODT xml file (.fodt)
5. `grep -Eo "https://www.nu.nl/[A-Za-z\-]+/[0-9]+/" [copied file].fodt|sort|uniq > [url file].txt`

wrt 3 and 4: this can also be done with Microsoft Word, open the `docx` file in a file extraction program (e.g., WinZip) and locate the relevant `.xml` file. Probably a selenium/xpath solution is also possible for step 3-5.

## code block 1: Initialization (always run)

In [1]:
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from IPython.display import clear_output
import pandas as pd
import re
import time

def id_from_article_url(article_url):
    res = re.search(r"([0-9]+)/", article_url)
    return res.group(1)

def category_from_article_url(article_url):
    res = re.search(r"^https://www.nu.nl/([^\/]+)/", article_url)
    return res.group(1)

def talk_url_from_id(article_id):
    return "https://talk.nu.nl/embed/stream?asset_url=https%3A%2F%2Fwww.nu.nl%2Fartikel%2F"+\
           str(article_id)+"%2Fredirect.html&initialWidth=601&childId=coral_talk_wrapper"

In [2]:
DRIVER = "/usr/bin/geckodriver"
DATADIR = "../data/nunl/"
URLFILE = "nunl-urls-20200913.csv"

urls_all = list(pd.read_csv(URLFILE,header=None)[0])

## code block 2: Download metadata: date, title, abstract, content

In [ ]:
import json
import csv
from selenium.webdriver.firefox.options import Options

In [ ]:
OUTFILEMETADONELIST = "nunl_meta_1.csv nunl_meta_2.csv nunl_meta_3.csv nunl_meta_4.csv".split()
URL = "url"

urlsDone = {}
for inFileName in OUTFILEMETADONELIST:
    dfDone = pd.read_csv(inFileName)
    for i in range(0,len(dfDone)): urlsDone[dfDone.iloc[i][URL]] = True

In [ ]:
OUTFILEMETA = 'nunl_meta.csv'
OUTFILENOMETA = 'nunl_nometa.txt'
PARBREAK = r" \n "

start_id = 0
write_mode = 'w'
if start_id > 0: write_mode = 'a'

options = Options()
options.headless = True

file_out = open(OUTFILEMETA, write_mode, encoding = 'utf8', newline = '')
meta_out = csv.writer(file_out,
                      delimiter = ',',
                      quotechar = '"',
                      doublequote = True,
                      quoting = csv.QUOTE_NONNUMERIC)
if start_id == 0:
    meta_out.writerow(['url','date','title','abstract','body','type'])

na_out = open(OUTFILENOMETA, 'a')

In [ ]:
for i in range(start_id,len(urls_all)):
    clear_output(wait=True)
    print(i, "/", len(urls_all)-1, urls_all[i])
    if urls_all[i] in urlsDone: continue
    driver = webdriver.Firefox(options=options, executable_path=DRIVER)
    try: 
        driver.get(urls_all[i])
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
        node_type = driver.find_element_by_xpath("//head/meta[@name='og:type']")
        txt_type = node_type.get_attribute('content')
        node_title = driver.find_element_by_xpath("//head/meta[@name='title']")
        txt_title = node_title.get_attribute('content')
        node_abstract = driver.find_element_by_xpath("//head/meta[@name='description']")
        txt_abstract = node_abstract.get_attribute('content')
        txt_abstract = re.sub("\n"," \\\\n ",txt_abstract)
        if "video" in txt_type:
            txt_type = "video"
            node_date = driver.find_element_by_xpath("//head/meta[@name='video:release_date']")
            txt_date = node_date.get_attribute('content')
            content = ""
            for paragraph in driver.find_element_by_xpath("//div[@class='caption']").find_elements_by_xpath(".//p"): 
                content += paragraph.text + PARBREAK
        else:
            txt_type = "article"
            node_date = driver.find_element_by_xpath("//head/meta[@name='article:published_time']")
            txt_date = node_date.get_attribute('content')
            content = ""
            for paragraph in driver.find_element_by_xpath("//div[@class='block article-body']").find_elements_by_xpath(".//p"): 
                content += paragraph.text + PARBREAK
        content = re.sub("\n"," \\\\n ",content)
        meta_out.writerow([urls_all[i], txt_date, txt_title, txt_abstract, content,txt_type])
        print(txt_date,txt_title)
        driver.quit()
        time.sleep(2)
    except: pass
file_out.close()
na_out.close()

## code block 3 (target url collection)

In [ ]:
# show all articles from Corona tag page
WEBPAGE = "https://www.nu.nl/tag/Coronavirus"
driver = webdriver.Firefox(executable_path=DRIVER)
driver.get(WEBPAGE)
driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")

Now click on **Laad meer artikelen** to load the links to all required articles

## code block 4 (target url collection, continued)

In [ ]:
# click 'show more articles' a few hundred times
# afterwards copy and paste to extract urls
for i in range(0):
    clear_output(wait=True)
    print(i)
    morearticles_link = driver.find_element_by_xpath("//a[starts-with(@class,'block-more__link')]")
    try:
        morearticles_link.click()
        time.sleep(1)
    except StaleElementReferenceException as e:
        # new xpath executed before DOM update from previous click finished
        print("stale element")
        break

In [ ]:
urls_all = []
elementsA = driver.find_elements_by_xpath("//a")
for elementA in elementsA:
    url = elementA.get_attribute('href')
    try:
        if len(re.findall("/",url)) > 4: urls_all.append(url)
    except: pass

In [ ]:
print(len(urls_all),urls_all[1],urls_all[-1],sep="\n")

In [ ]:
URLFILE = "nunl-urls-20201002.csv"
pd.DataFrame(urls_all).drop_duplicates().to_csv(URLFILE,header=None,index=None)

## code block 5 (skip)

In [ ]:
# show all reactions for single page
WEBPAGE = "https://www.nu.nl/coronavirus/6055448/ggd-deed-ruim-9000-coronatesten-sinds-maandag-tot-nu-toe-94-positief.html" 
WEBPAGE ="https://talk.nu.nl/embed/stream?asset_url=https%3A%2F%2Fwww.nu.nl%2Fartikel%2F6055448%2Fredirect.html&initialWidth=601&childId=coral_talk_wrapper&parentTitle=GGD%20deed%20ruim%209.000%20coronatesten%20sinds%20maandag%2C%20tot%20nu%20toe%2094%20positief%20%7C%20NU%20-%20Het%20laatste%20nieuws%20het%20eerst%20op%20NU.nl&parentUrl=https%3A%2F%2Fwww.nu.nl%2Fcoronavirus%2F6055448%2Fggd-deed-ruim-9000-coronatesten-sinds-maandag-tot-nu-toe-94-positief.html%23coral_talk_wrapper"

driver = webdriver.Firefox(executable_path=DRIVER)
driver.get(WEBPAGE)

morereactions = True
while morereactions:
    reaction_buttons = driver.find_elements_by_xpath("//button[contains(text(),'meer reacties')]")
    print("number of buttons:", len(reaction_buttons))
    if len(reaction_buttons) > 0:
        python_button = reaction_buttons[0]
        try:
            python_button.click()
        except StaleElementReferenceException as e:
            # new xpath executed before DOM update from previous click finished
            pass
    else:
        morereactions = False

## code block 6 (Get comments)

In [3]:
DATADIROUT = "../data/nunl/downloads4/"
URLFILE1 = "nunl-urls-20200724.csv"
URLFILE2 = "nunl-urls-20200726.csv"
URLFILE3 = "nunl-urls-20200807.csv"
URLFILE4 = "nunl-urls-20200913.csv"
URLFILE = "nunl-urls-20201002.csv"

urls_all = list(pd.read_csv(URLFILE,header=None)[0])

In [5]:
import os

COMMENT = r"^Comment__commentContainer"
AUTHORNAME = r"^AuthorName__name"
TIMESTAMP = r"^CommentTimestamp__timestamp"
TEXT = r"^Comment__content"
CLASS = "class"
TITLE = "title"
XPATHID = "../../.."
XPATHPARENTID = "../../../../.."
WAIT = 10

for current_id in range(0,len(urls_all)):
    current_url = urls_all[current_id]
    
    print(current_id, "/", len(urls_all)-1, current_url, "id:", id_from_article_url(current_url), 
          "category:", category_from_article_url(current_url))
    
    article_id = id_from_article_url(current_url)
    article_cat = category_from_article_url(current_url)
    
    outputFile = DATADIROUT+f"{article_cat}-{article_id}.csv"
    if os.path.exists(outputFile): continue
    pd.DataFrame({}).to_csv(outputFile)
    
    webPage = talk_url_from_id(article_id)
    
    driver = webdriver.Firefox(executable_path=DRIVER)
    driver.get(webPage)
    driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
    
    # wait for spinner to finish
    while len(driver.find_elements_by_xpath("//div[starts-with(@class, 'Spinner__container')]")) != 0:
        time.sleep(1)
    
    # click more reactions button until all reactions are shown
    print(f"click more reaction buttons") 
    morereactions = True
    while morereactions:
        reaction_buttons = driver.find_elements_by_xpath("//button[contains(text(),'meer reacties')]")
        print(f"number of buttons:", len(reaction_buttons))
        if len(reaction_buttons) > 0:
            python_button = reaction_buttons[0]
            pd.DataFrame({}).to_csv(outputFile+".squeal")
            try:
                python_button.click()
                time.sleep(WAIT)
            except StaleElementReferenceException as e:
                # new xpath executed before DOM update from previous click finished
                pass
        else:
            morereactions = False
            print("no more reactions")
    
    # wait for spinner to finish
    while len(driver.find_elements_by_xpath("//div[starts-with(@class, 'Spinner__container')]")) != 0:
        time.sleep(1)
    
    # locate all reactions and store in csv file
    print(f"locate all reactions")  
    data = []
    alldivs = driver.find_elements_by_xpath("//div[starts-with(@class, 'Comment__commentContainer')]")
    divnr = 0
    lenall = len(alldivs)
    for e in alldivs:
        divnr += 1
        eClass = e.get_attribute(CLASS)
        if re.search(COMMENT,eClass):
            eId = e.find_element_by_xpath(XPATHID).id
            parent = e.find_element_by_xpath(XPATHPARENTID).id
            authorName = ""
            timeStamp = ""
            text = ""
            for f in e.find_elements_by_xpath(".//*"):
                fClass = f.get_attribute(CLASS)
                if fClass != eClass:
                    if re.search(AUTHORNAME,fClass): 
                        authorName = f.text
                    elif re.search(TIMESTAMP,fClass): 
                        timeStamp = f.get_attribute(TITLE)
                    elif re.search(TEXT,fClass): 
                        text = re.sub("\n"," ",f.text)
                        break
            data.append({"id":eId,"name":authorName,"date":timeStamp,"text":text,"parent":parent})
            clear_output(wait=True)
            print(current_id, "/", len(urls_all), current_url, "id:", id_from_article_url(current_url), "category:", category_from_article_url(current_url))
            print(f"processed: {len(data)} ({divnr} / {lenall} divs)")
    
    df = pd.DataFrame(data)
    df.to_csv(outputFile,index=False)
    driver.quit()
    print("finished processing", current_url)

246 / 365 https://www.nu.nl/coronavirus/6077051/klinische-proeven-oxford-vaccin-astrazeneca-weer-van-start.html id: 6077051 category: coronavirus
processed: 728 (728 / 728 divs)
finished processing https://www.nu.nl/coronavirus/6077051/klinische-proeven-oxford-vaccin-astrazeneca-weer-van-start.html
247 / 364 https://www.nu.nl/coronavirus/6076959/ggd-haaglanden-legt-contactonderzoek-deels-in-handen-van-coronapatient.html id: 6076959 category: coronavirus
248 / 364 https://www.nu.nl/coronavirus/6076949/van-dissel-regionale-maatregelen-nodig-als-besmettingen-blijven-stijgen.html id: 6076949 category: coronavirus
249 / 364 https://www.nu.nl/coronavirus/6076638/deel-leraren-en-zorgpersoneel-krijgt-tijdelijk-voorrang-bij-coronatest.html id: 6076638 category: coronavirus
250 / 364 https://www.nu.nl/weekend/6076585/de-jacht-op-een-coronavaccin-zo-verloopt-vaccinonderzoek.html id: 6076585 category: weekend
251 / 364 https://www.nu.nl/dit-wordt-het-nieuws/6076842/tourbubbel-op-rustdag-getest-op-

## code block 7 (target urls save)

In [ ]:
URLFILE = "nunl-urls-20200726.csv"

nunl_urls = []
for current_id in range(0,len(urls_all)):
    current_url = urls_all[current_id]
    article_id = id_from_article_url(current_url)
    article_cat = category_from_article_url(current_url)
    
    outputFile = f"nunl-{article_cat}-{article_id}-{current_id}.csv"
    nunl_urls.append((current_url,outputFile))

pd.DataFrame(nunl_urls).to_csv(URLFILE)
df = pd.read_csv(URLFILE,index_col=0)
df.iloc[:10]

In [ ]:
"economie-6066366-0.csv" in list(df["1"])

## Test code

In [ ]:
if True:
    # wait for spinner to finish
    while len(driver.find_elements_by_xpath("//div[starts-with(@class, 'Spinner__container')]")) != 0:
        time.sleep(1)
    
    # locate all reactions and store in csv file
    print("locate all reactions")  
    data = []
    alldivs = driver.find_elements_by_xpath("//div")
    divnr = 0
    lenall = len(alldivs)
    for e in alldivs:
        divnr += 1
        eClass = e.get_attribute(CLASS)
        print(eClass)
        if re.search(COMMENT,eClass):
            print("comment found")
            eId = e.find_elements_by_xpath(XPATHID)[0].id
            parent = e.find_elements_by_xpath(XPATHPARENTID)[0].id
            authorName = ""
            timeStamp = ""
            text = ""
            for f in e.find_elements_by_xpath(".//*"):
                fClass = f.get_attribute(CLASS)
                if fClass != eClass:
                    if re.search(AUTHORNAME,fClass): 
                        authorName = f.text
                    elif re.search(TIMESTAMP,fClass): 
                        timeStamp = f.get_attribute(TITLE)
                    elif re.search(TEXT,fClass): 
                        text = re.sub("\n"," ",f.text)
                        break
            data.append({"id":eId,"name":authorName,"date":timeStamp,"text":text,"parent":parent})
            clear_output(wait=True)
            print(current_id, "/", len(urls_all), current_url, "id:", id_from_article_url(current_url), "category:", category_from_article_url(current_url))
            print(f"processed: {len(data)} ({divnr} / {lenall} divs)")
    
    df = pd.DataFrame(data)
    df.to_csv(outputFile,index=False)
    #driver.quit()
    print("finished processing", current_url)

In [ ]:
print("test")

In [ ]:
if True:
    current_url = urls_all[current_id]
    
    print(current_id, "/", len(urls_all)-1, current_url, "id:", id_from_article_url(current_url), "category:", category_from_article_url(current_url))
    
    article_id = id_from_article_url(current_url)
    article_cat = category_from_article_url(current_url)
    
    outputFile = f"{article_cat}-{article_id}-{current_id}.csv"
    
    webPage = talk_url_from_id(article_id)
    
    driver = webdriver.Firefox(executable_path=DRIVER)
    driver.get(webPage)
    driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
